In [3]:
import sys
sys.path.append('/home/mila/l/leo.gagnon/latent_control')

%load_ext autoreload
%autoreload 2
from lightning_modules.diffusion_prior import DiffusionPriorTask
import torch
import matplotlib.pyplot as plt
from data.diffusion import LatentDiffusionDataset
from torch2jax import j2t, t2j
import jax.numpy as jnp
import jax
from jax.scipy.special import rel_entr

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/home/mila/l/leo.gagnon/latent_control/venv/lib/python3.10/site-packages/pl_bolts/__init__.py:11: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(numpy, tp_name):
/home/mila/l/leo.gagnon/latent_control/venv/lib/python3.10/site-packages/pl_bolts/__init__.py:11: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  if not hasattr(numpy, tp_name):
/home/mila/l/leo.gagnon/latent_control/venv/lib/python3.10/site-packages/pl_bolts/models/self_supervised/amdim/amdim_module.py:34: UnderReviewWarning: The feature generate_power_seq is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality may change without warning in future releases. More details: https://lightning-bolts.readthedocs.io/en/latest/stability.html
  "lr_options": generate_power_seq(LEARNING_RATE_CIFAR, 11),
/home/mila/l/leo.gagnon/late

In [4]:
task = DiffusionPriorTask('8dy9p44k')
task.cuda();

Loaded dataset : (11288/1000)
Loaded checkpoing : last.ckpt
Loaded checkpoing : last.ckpt


In [13]:
task.base_task.model.device

AttributeError: 'MetaLearner' object has no attribute 'device'

In [1]:
dataset = task.train_data.dataset 
dataset: LatentDiffusionDataset

NameError: name 'task' is not defined

In [124]:
raw_latent, cond_input_ids, cond_ignore_mask, cond_tokens, latent = dataset[1090].values()

/home/mila/l/leo.gagnon/latent_control/data/hmm.py:608: FutureWarning: None encountered in jnp.array(); this is currently treated as NaN. In the future this will result in an error.
  intv_envs = jnp.array(intv_envs)


In [125]:
xd = 8
cond_mask = torch.BoolTensor([True]*xd + [False]*(50-xd))[None].cuda()

In [126]:
task.diffusion_prior.cfg.sampler = 'ddpm' # ddpm, dpmpp, ddim
task.diffusion_prior.cfg.sampling_timesteps=100

In [147]:
z_t = task.diffusion_prior.sample(250, cond=cond_tokens, cond_mask=cond_mask, cls_free_guidance=1.5)
if task.diffusion_prior.cfg.normalize_latent:
    z_t = task.diffusion_prior.unnormalize_latent(z_t)

sampling loop time step:   0%|          | 0/100 [00:00<?, ?it/s]

In [148]:
diff_dataset = task.train_data.dataset

In [149]:
decoded_task_latent = [
    torch.Tensor(
        [
            (latent_embds.weight @ sampled_latent.T).argmax()
            for latent_embds in diff_dataset.known_encoder.latent_embedding
        ]
    )
    for sampled_latent in z_t
]
decoded_task_latent = torch.stack(decoded_task_latent, 0)

In [150]:
decoded_task_id = jnp.stack(
    [
        (
            task.base_task.full_data.index_to_latent
            == t2j(decoded_task_latent[i])
        )
        .all(-1)
        .argmax()
        for i in range(len(decoded_task_latent))
    ]
)

In [151]:
empirical_dist = jnp.bincount(decoded_task_id, minlength=len(task.base_task.full_data))
empirical_dist = empirical_dist/empirical_dist.sum()

In [152]:
oracle_dist = task.base_task.full_data.bayesian_oracle(
    jnp.arange(len(task.base_task.full_data)),
    t2j(cond_input_ids[0]),
)['log_alpha_post']

In [153]:
for id in empirical_dist.nonzero()[0]:
    print(f'HMM {task.base_task.full_data.index_to_latent[id]}, #{id}  : {empirical_dist[id]}')

HMM [0 0 1 0 0 1 0 1 0 1 1 1], #898  : 0.20000000298023224
HMM [0 0 1 0 1 1 0 1 0 1 1 1], #1090  : 0.800000011920929


In [154]:
oracle_ = jnp.exp(oracle_dist[cond_mask.int().argmin().item()+1]).round(2).to_device(jax.devices('cpu')[0])
for id in oracle_.nonzero()[0]:
    print(f'HMM #{id}, {task.base_task.full_data.index_to_latent[id]} : {oracle_[id]}')

HMM #124, [0 0 0 0 0 1 0 1 0 0 1 1] : 0.019999999552965164
HMM #130, [0 0 0 0 0 1 0 1 0 1 1 1] : 0.009999999776482582
HMM #898, [0 0 1 0 0 1 0 1 0 1 1 1] : 0.019999999552965164
HMM #946, [0 0 1 0 0 1 1 1 0 1 1 1] : 0.05999999865889549
HMM #994, [0 0 1 0 1 0 0 1 0 1 1 1] : 0.009999999776482582
HMM #1090, [0 0 1 0 1 1 0 1 0 1 1 1] : 0.05999999865889549
HMM #1138, [0 0 1 0 1 1 1 1 0 1 1 1] : 0.05999999865889549
HMM #5410, [1 1 1 0 0 0 0 1 0 1 1 1] : 0.009999999776482582
HMM #5500, [1 1 1 0 0 1 0 1 0 0 1 1] : 0.08999999612569809
HMM #5506, [1 1 1 0 0 1 0 1 0 1 1 1] : 0.14000000059604645
HMM #5602, [1 1 1 0 1 0 0 1 0 1 1 1] : 0.009999999776482582
HMM #5692, [1 1 1 0 1 1 0 1 0 0 1 1] : 0.04999999701976776
HMM #5698, [1 1 1 0 1 1 0 1 0 1 1 1] : 0.14000000059604645
HMM #6178, [2 0 0 0 0 0 0 1 0 1 1 1] : 0.009999999776482582
HMM #6274, [2 0 0 0 0 1 0 1 0 1 1 1] : 0.04999999701976776
HMM #6370, [2 0 0 0 1 0 0 1 0 1 1 1] : 0.009999999776482582
HMM #6466, [2 0 0 0 1 1 0 1 0 1 1 1] : 0.029999999329

In [145]:
empirical_dist_ = empirical_dist + 1e-8
empirical_dist_ = empirical_dist_/empirical_dist_.sum()
rel_entr(oracle_, empirical_dist_).sum()

Array(12.6755495, dtype=float32)

In [146]:
oracle__ = oracle_ + 1e-8
oracle__ = oracle__/oracle__.sum()
rel_entr(empirical_dist, oracle__).sum()

Array(11.322886, dtype=float32)

In [231]:
emb = nn.ModuleList(
    [
        nn.Embedding(
            n,
            512,
        )
        for n in [5,2,2,3]
    ]
)

In [233]:
directions = orthogonal_(torch.zeros((512,512)))

i = 0
for e in emb:
    weight_ = torch.zeros_like(e.weight)
    for j in range(e.weight.shape[0]):
        weight_[j] = directions[i]
        i += 1
    e.weight = nn.Parameter(weight_)

In [234]:
torch.dot(emb[0].weight[0],emb[0].weight[2])

tensor(-3.0734e-08, grad_fn=<DotBackward0>)

In [217]:
emb[0].weight.shape

torch.Size([5, 512])

In [210]:
torch.dot(a[:,0], a[:,5])

tensor(-8.3819e-09)

In [88]:
true = torch.Tensor(
    [
        (latent_embds.weight @ latent[0].T).argmax()
        for latent_embds in task.base_task.model.encoder.latent_embedding
    ]
)

In [86]:
print(f'Accuracy : {torch.mean(torch.sum(preds == true[None],-1)/len(true))}')

Accuracy : 0.6158332824707031


In [95]:
(torch.sum(preds == true[None],0)/100).mean(0)

tensor(0.6158)

In [93]:
len(preds)

100